In [ ]:
! pip install evaluate

In [ ]:
from importlib import util
import os
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/campusx-team/Text-Datasets/refs/heads/main/Reddit_Data.csv')
df.dropna(inplace=True)
df.index.name='id'

In [ ]:
df.head(5)

,clean_comment,category
id,,
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
# Rename the Feature
df.rename(columns={'clean_comment':'text'},inplace=True)

In [ ]:
df.category.value_counts()

,count
category,
1,15830
0,13042
-1,8277


In [ ]:
# Fetching the unique elements from category feature
possible_labels=df.category.unique()
possible_labels.sort()

In [ ]:
# Labeling the category elements
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [ ]:
# Adding the new feature
df['label'] = df.category.replace(label_dict)

In [ ]:
df.head()

,text,category,label
id,,,
0,family mormon have never tried explain them t...,1,2
1,buddhism has very much lot compatible with chr...,1,2
2,seriously don say thing first all they won get...,-1,0
3,what you have learned yours and only yours wha...,0,1
4,for your own benefit you may want read living ...,1,2


In [ ]:
X_train,X_val,y_train,y_val= train_test_split(df.index.values,df.label.values,test_size=0.15,random_state=24,stratify=df.label.values)


In [ ]:
df['data_type']= ['not_set']* df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'label', 'data_type']).count()

text
category label data_type       
-1       0     train       7035
               val         1242
 0       1     train      11086
               val         1956
 1       2     train      13455
               val         2375

In [ ]:
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels= len(label_dict),output_attentions=False, output_hidden_states=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
encoded_data_train=tokenizer.batch_encode_plus(df[df.data_type=='train'].text.values, add_special_tokens=True, return_attention_mask=True, max_length=256, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).